In [1]:
from scipy.io import loadmat
import numpy as np
import math, itertools, copy
mat_variables = loadmat("matlab_param/test_5_5_2.mat")

In [2]:
def manhattan(p1, p2):
    ''' Manhattan distance'''
    x = abs(p1[0] - p2[0])**2
    y = abs(p1[1] - p2[1])**2
    return math.sqrt(x+y)

In [9]:
class Route:
    def __init__(self,idx,route):
        self.id = idx
        self.x, self.y, self.z = self.extractRoute(route)
        self.length = len(self.x)

    def extractRoute(self, route):
        x = route['x'][0][0][0][:]
        y = route['y'][0][0][0][:]
        z = route['z'][0][0][0][:]
        x_list, y_list, z_list = [],[],[]
        for j in range(len(x)):
            x_list.append(x[j])
            y_list.append(y[j])
            z_list.append(z[j])
        return x_list,y_list,z_list

class World:
    def __init__(self, fileName):
        # Global variable
        mat_variables = loadmat(fileName)
        self.map = mat_variables['map']
        self.numDrones = mat_variables['numRobots'][0][0]
        self.routes = [Route(i,mat_variables['routes'][0][i]) for i in range(self.numDrones)]
        self.maxLength = self.findMaxLength()
        # self.roots = [i[0].tolist() for i in mat_variables['R4'][0]]
        self.roots = [[self.routes[i].x[0], self.routes[i].y[0], self.routes[i].z[0]] for i in range(self.numDrones)]

        # Technical parameters
        self.z_land = 0.2 # target for landing
        self.z_target = 1 # normal fly altitude
        self.z_offset = 0.3 # altitude offset when conflicts
        
        # Solve conflicts
        # self.solveConflicts()

    def findMaxLength(self):
        ''' Find the path with maximum length '''
        temp = 0
        for i in range(len(self.routes)):
            max_i = max(len(self.routes[i].x),len(self.routes[i].y),len(self.routes[i].z))
            if max_i > temp:
                temp = max_i
        return temp

    def adjustLength(self):
        ''' Make paths of same length -> when occurred, bring the drones to land altitude'''
        for r in range(self.numDrones):
            while len(self.routes[r].x) < self.maxLength:
                self.routes[r].x.append(self.routes[r].x[-1]) # x: append last element
                self.routes[r].y.append(self.routes[r].y[-1]) # y: append last element
                self.routes[r].z.append(self.z_land) # z: append z land
            self.routes[r].length = self.maxLength

    def checkNextXYloc(self,k,idx):
        ''' Check conflict in only in x and y -> used when we changed altitude '''
        # k: next drone
        # idx: index next step in path
        for i in range(self.numDrones):
            if i != k:
                if (self.routes[k].x[idx] == self.routes[i].x[idx]
                    and self.routes[k].y[idx] == self.routes[i].y[idx]):
                    return True
        return False
    
    def conflictXY(self,j,l,idx):
        ''' Check conflict in X Y'''
        # j: current drone
        # l: next drone
        # idx: current index in path
        if (self.routes[j].x[idx] == self.routes[l].x[idx] 
            and self.routes[j].y[idx] == self.routes[l].y[idx]):
            return True
        else: return False
    
    def conflictXYZ(self,j,k,idx):
        ''' Check conflict in X Y Z '''
        # j: current drone
        # k: next drone
        # idx: current index in path
        if (self.routes[j].x[idx] == self.routes[k].x[idx]
            and self.routes[j].y[idx] == self.routes[k].y[idx]
            and self.routes[j].z[idx] == self.routes[k].z[idx]):
            return True
        else: return False

    def insertWait(self,id,idx):
        ''' Wait for 1 step'''
        # l: drone
        self.routes[id].x.insert(idx,self.routes[id].x[i-1])
        self.routes[id].y.insert(idx,self.routes[id].y[i-1])
        self.routes[id].z.insert(idx,self.routes[id].z[i-1])

    def copyLastZ(self,idx):
        ''' Copy value of previous Z for all drones'''
        for i in range(self.numDrones):
            if self.routes[i].z[idx] != self.z_land:
                self.routes[i].z[idx] = self.routes[i].z[idx-1]

    def setZtoTarget(self,idx,solvedConflict):
        ''' Bring back the drones to the z_target when possible'''
        for i in range(len(self.routes)):
            if i not in solvedConflict:
                if self.routes[i].z[idx] != self.z_land:
                    self.routes[i].z[idx] = self.z_target

    def countConflictXY(self,j,idx):
        ''' Count number of conflict between the current and the following drones'''
        droneID = []
        for i in range(j+1,self.numDrones):
            if self.conflictXY(j, i, idx):
                droneID.append(i)
        return droneID
        
    def solveConflicts(self):
        ''' Solve conflicts until no new steps are added'''
        self.adjustLength()
        prev_length = self.maxLength
        while True:
            for i in range(1,self.maxLength):

                # Copy the last Z for all drones
                self.copyLastZ(i)
                # Keep track of the conflict solved
                solvedConflict = []

                for j in range(self.numDrones):
                    # Z value of current drone
                    temp1 = self.routes[j].z[i]

                    # Find drone in conflict
                    droneInConflict = self.countConflictXY(j,i)
                    if droneInConflict:
                        solvedConflict.insert(0,j)
                    for el in droneInConflict:
                        if el in solvedConflict:
                            droneInConflict.remove(el)
                        else:
                            solvedConflict.append(el)
            
                    # Solve conflicts
                    for count in range(len(droneInConflict)):
                        if temp1 > self.z_target:
                            self.routes[droneInConflict[count]].z[i] = round(temp1 - (count+1)*self.z_offset,1)
                        elif temp1 < self.z_target:
                            self.routes[droneInConflict[count]].z[i] = round(temp1 + (count+1)*self.z_offset,1)
                        # If more thant 3 drones want to access the same cell, from the 4-th and on they have to wait
                        elif count > 1:
                            self.insertWait(droneInConflict[count],i)
                        elif count % 2 == 0: # temp1 == z_target
                            self.routes[droneInConflict[count]].z[i] = round(temp1 + self.z_offset,1)
                        else:
                            self.routes[droneInConflict[count]].z[i] = round(temp1 - self.z_offset,1)
            
                self.setZtoTarget(i,solvedConflict)

            # Check length
            self.adjustLength()
            self.maxLength = self.findMaxLength()
            if prev_length == self.maxLength:
                break

In [4]:
param = World("matlab_param/test_5_5_2.mat")
param.roots

[[0.5, 2.5, 1], [2.5, 4.5, 1]]

In [12]:
pair1 = [[0, 0],[0, 1]]
pair2 = param.roots
pairs = [list(zip(x,pair1)) for x in itertools.permutations(pair2,len(pair1))]
def noConflictPairs(pairs):
    ''' Assign each drones to path, based on the initial position on the path'''
    minD = 100000000
    for i in range(len(pairs)):
        pairedDist = 0
        for j in range(len(pairs[i])):
            p1 = pairs[i][j][0]
            p2 = pairs[i][j][1]
            pairedDist += manhattan(p1,p2)
        # print(pairedDist)
        if pairedDist < minD:
            minD = pairedDist
            bestIdx = i
        print(pairs[i],pairedDist)
    return pairs[bestIdx]


pair = noConflictPairs(pairs)
print(pair)
index = [pair2.index(el[0]) for el in pair]
# print(pair)
print(index)

# param1 = copy.deepcopy(param)
# for i in range(len(index)):
#     param1.routes[i] = param.routes[index[i]]
print("----------")
param1 = [param.routes[index[i]] for i in range(param.numDrones)]
for el in param1:
    print("x: ",el.x)
    print("y: ",el.y)
    print("z: ",el.z)
    print("\n")


([([0.5, 2.5, 1], [0, 0]), ([2.5, 4.5, 1], [0, 1])], 6.850672390317706)
([([2.5, 4.5, 1], [0, 0]), ([0.5, 2.5, 1], [0, 1])], 6.72895390057769)
[([2.5, 4.5, 1], [0, 0]), ([0.5, 2.5, 1], [0, 1])]
[1, 0]
----------
('x: ', [2.5, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.0, 4.5, 4.0, 3.5, 3.0, 3.0, 3.5, 4.0, 4.5, 5.0, 5.0, 4.5, 4.0, 3.5, 3.0, 3.0, 3.0, 3.0, 3.5, 4.0, 4.0, 3.5, 3.0, 2.5, 2.0, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.0, 4.5, 4.0, 3.5, 3.0, 2.5, 2.0, 1.5, 1.5, 1.5, 1.0, 0.5, 0.5, 1.0, 1.5, 2.0, 2.5, 2.5, 2.5, 2.5, 2.5])
('y: ', [4.5, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.5, 4.5, 4.5, 4.5, 4.5, 4.0, 4.0, 4.0, 4.0, 4.0, 3.5, 3.5, 3.5, 3.5, 3.5, 3.0, 2.5, 2.0, 2.0, 2.0, 1.5, 1.5, 1.5, 1.5, 1.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1.0, 1.5, 1.5, 1.5, 2.0, 2.0, 2.0, 2.0, 2.0, 2.5, 3.0, 3.5, 4.0])
('z: ', [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [13]:
for el in param.routes:
    print("x: ",el.x)
    print("y: ",el.y)
    print("z: ",el.z)
    print("\n")

('x: ', [0.5, 0.5, 0.5, 0.5, 1.0, 1.5, 2.0, 2.0, 1.5, 1.0, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.0, 4.5, 4.0, 3.5, 3.0, 2.5, 2.0, 1.5, 1.0])
('y: ', [2.5, 3.0, 3.5, 4.0, 4.0, 4.0, 4.0, 3.5, 3.5, 3.5, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5])
('z: ', [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


('x: ', [2.5, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.0, 4.5, 4.0, 3.5, 3.0, 3.0, 3.5, 4.0, 4.5, 5.0, 5.0, 4.5, 4.0, 3.5, 3.0, 3.0, 3.0, 3.0, 3.5, 4.0, 4.0, 3.5, 3.0, 2.5, 2.0, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.0, 4.5, 4.0, 3.5, 3.0, 2.5, 2.0, 1.5, 1.5, 1.5, 1.0, 0.5, 0.5, 1.0, 1.5, 2.0, 2.5, 2.5, 2.5, 2.5, 2.5])
('y: ', [4.5, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.5, 4.5, 4.5, 4.5, 4.5, 4.0, 4.0, 4.0, 4.0, 4.0, 3.5, 3.5, 3.5, 3.5, 3.5, 3.0, 2.5, 2.0, 2.0, 2.0, 1.5, 1.5, 1.5, 1.5, 1.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1.0, 1.5, 1.5, 1.5, 2.0, 2.0, 2.0, 2.0, 2.0, 2.5, 

In [17]:

np.array(param.roots.pop())

array([2.5, 4.5, 1. ])

[(2.5, 4.5, 1), (0.5, 2.5, 1)]

In [14]:
pairs = [list(zip(x,pair1)) for x in itertools.permutations(param.roots,param.numDrones)]
# Find the one with lowest distance
pair = noConflictPairs(pairs)
# Find index to reorder the routes according to the pair found
index = [param.roots.index(el[0]) for el in pair]
newRoutes = copy.deepcopy(param.routes)
for i in range(len(index)):
    newRoutes[i] = param.routes[index[i]]
param.routes = copy.deepcopy(newRoutes)
for el in param.routes:
    print("x: ",el.x)
    print("y: ",el.y)
    print("z: ",el.z)
    print("\n")


([([0.5, 2.5, 1], [0, 0]), ([2.5, 4.5, 1], [0, 1])], 6.850672390317706)
([([2.5, 4.5, 1], [0, 0]), ([0.5, 2.5, 1], [0, 1])], 6.72895390057769)
('x: ', [2.5, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.0, 4.5, 4.0, 3.5, 3.0, 3.0, 3.5, 4.0, 4.5, 5.0, 5.0, 4.5, 4.0, 3.5, 3.0, 3.0, 3.0, 3.0, 3.5, 4.0, 4.0, 3.5, 3.0, 2.5, 2.0, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.0, 4.5, 4.0, 3.5, 3.0, 2.5, 2.0, 1.5, 1.5, 1.5, 1.0, 0.5, 0.5, 1.0, 1.5, 2.0, 2.5, 2.5, 2.5, 2.5, 2.5])
('y: ', [4.5, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.5, 4.5, 4.5, 4.5, 4.5, 4.0, 4.0, 4.0, 4.0, 4.0, 3.5, 3.5, 3.5, 3.5, 3.5, 3.0, 2.5, 2.0, 2.0, 2.0, 1.5, 1.5, 1.5, 1.5, 1.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1.0, 1.5, 1.5, 1.5, 2.0, 2.0, 2.0, 2.0, 2.0, 2.5, 3.0, 3.5, 4.0])
('z: ', [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


('x: ', [0.5, 0.5, 0.5, 0.5, 1.0, 1.5, 2.0, 2

In [25]:
goal = [(param.routes[cf].x[0], param.routes[cf].y[0], param.routes[cf].z[0]) for cf in range(param.numDrones)]
np.array(goal[0]).astype(float)

array([2.5, 4.5, 1. ])

In [ ]:
def conflictZ(routes,j,l,idx):
    ''' Check conflict in X Y'''
    # j: current drone
    # l: next drone
    # idx: current index in path
    if routes[str(j)][2][idx] == routes[str(l)][2][idx]:
        return True
    else: return False

def conflictXY(routes,j,l,idx):
    ''' Check conflict in X Y'''
    # j: current drone
    # l: next drone
    # idx: current index in path
    if (routes[str(j)][0][idx] == routes[str(l)][0][idx] 
        and routes[str(j)][1][idx] == routes[str(l)][1][idx]):
        return True
    else: return False

def conflictXYZ(routes,j,k,idx):
    ''' Check conflict in X Y Z '''
    # j: current drone
    # k: next drone
    # idx: current index in path
    if (routes[str(j)][0][idx] == routes[str(k)][0][idx] 
        and routes[str(j)][1][idx] == routes[str(k)][1][idx]
        and routes[str(j)][2][idx] == routes[str(k)][2][idx]):
        return True
    else: return False

def checkNextXYLoc(routes,k,idx):
    ''' Check conflict in only in x and y -> used when we changed altitude '''
    # k: next drone
    # idx: index next step in path
    for i in range(len(routes)):
        if i != k:
            if conflictXY(routes,k,i,idx):
                return True
    return False


def copyLastZ(routes,idx):
    for i in range(len(routes)):
        if routes[str(i)][2][idx] != z_land:
            routes[str(i)][2][idx] = routes[str(i)][2][idx-1]
    return routes

def setZtoTarget(routes,idx,solvedConflict):
    for i in range(len(routes)):
        if i not in solvedConflict:
            if routes[str(i)][2][idx] != z_land:
                routes[str(i)][2][idx] = z_target
    return routes

def countConflictXY(routes,j,idx):
    droneID = []
    for i in range(j+1,numDrones):
        if conflictXY(routes, j, i, idx):
            droneID.append(i)
    return droneID

z_land = 0.2 # target for landing
z_target = 1 # normal fly altitude
z_offset = 0.3 # altitude offset when conflicts


routes = {}
routes['0'] = [[1,1,3,5,3], [2,2,2,2,2], [z_target,z_target,z_target,z_target,z_target]]
routes['1'] = [[2,1,3,6,5], [2,2,2,2,2], [z_target,z_target,z_target,z_target,z_target]]
routes['2'] = [[3,1,4,7,5], [2,2,2,2,2], [z_target,z_target,z_target,z_target,z_target]]
routes['3'] = [[4,1,4,1,3], [2,2,2,2,2], [z_target,z_target,z_target,z_target,z_target]]
# routes['4'] = [[5,1,1,1], [2,2,2,2], [3,3,3,3,]]
maxLength = len(routes['0'][0])
numDrones = len(routes)
for r in routes:
    print(routes[r])

# Solve conflicts
for i in range(1,maxLength):  
    # Copy the last Z for all drones
    routes = copyLastZ(routes,i)
    # Keep track of the conflict solved
    solvedConflict = []

    for j in range(numDrones):
        # Z value of current drone
        temp1 = routes[str(j)][2][i]

        # Find drone in conflict
        droneInConflict = countConflictXY(routes,j,i)
        if droneInConflict:
            solvedConflict.insert(0,j)
        for el in droneInConflict:
            if el in solvedConflict:
                droneInConflict.remove(el)
            else:
                solvedConflict.append(el)

        # Solve conflicts
        for count in range(len(droneInConflict)):
            if temp1 > z_target:
                routes[str(droneInConflict[count])][2][i] = round(temp1 - (count+1)*z_offset,1)
            elif temp1 < z_target:
                routes[str(droneInConflict[count])][2][i] = round(temp1 + (count+1)*z_offset,1)
            # If more thant 3 drones want to access the same cell, from the 4-th and on they have to wait
            elif count > 1:
                routes[str(droneInConflict[count])][0].insert(i,routes[str(droneInConflict[count])][0][i-1])
                routes[str(droneInConflict[count])][1].insert(i,routes[str(droneInConflict[count])][1][i-1])
                routes[str(droneInConflict[count])][2].insert(i,routes[str(droneInConflict[count])][2][i-1])
            elif count % 2 == 0: # temp1 == z_target
                routes[str(droneInConflict[count])][2][i] = round(temp1 + z_offset,1)
            else:
                routes[str(droneInConflict[count])][2][i] = round(temp1 - z_offset,1)
    routes = setZtoTarget(routes,i,solvedConflict)
routes

In [ ]:
maxLength = len(routes['3'][0])
numDrones = len(routes)


for r in range(numDrones):
    while len(routes[str(r)][0]) < maxLength:
        routes[str(r)][0].append(routes[str(r)][0][-1]) # x: append last element
        routes[str(r)][1].append(routes[str(r)][1][-1]) # y: append last element
        routes[str(r)][2].append(z_land) # z: append z land
for r in routes:
    print(routes[r])

In [ ]:
# Solve conflicts
for i in range(1,maxLength):  
    # Copy the last Z for all drones
    routes = copyLastZ(routes,i)
    # Keep track of the conflict solved
    solvedConflict = []

    for j in range(numDrones):
        # Z value of current drone
        temp1 = routes[str(j)][2][i]

        # Find drone in conflict
        droneInConflict = countConflictXY(routes,j,i)
        if droneInConflict:
            solvedConflict.insert(0,j)
        for el in droneInConflict:
            if el in solvedConflict:
                droneInConflict.remove(el)
            else:
                solvedConflict.append(el)

        # Solve conflicts
        for count in range(len(droneInConflict)):
            if temp1 > z_target:
                routes[str(droneInConflict[count])][2][i] = round(temp1 - (count+1)*z_offset,1)
            elif temp1 < z_target:
                routes[str(droneInConflict[count])][2][i] = round(temp1 + (count+1)*z_offset,1)
            # If more thant 3 drones want to access the same cell, from the 4-th and on they have to wait
            elif count > 1:
                routes[str(droneInConflict[count])][0].insert(i,routes[str(droneInConflict[count])][0][i-1])
                routes[str(droneInConflict[count])][1].insert(i,routes[str(droneInConflict[count])][1][i-1])
                routes[str(droneInConflict[count])][2].insert(i,routes[str(droneInConflict[count])][2][i-1])
            elif count % 2 == 0: # temp1 == z_target
                routes[str(droneInConflict[count])][2][i] = round(temp1 + z_offset,1)
            else:
                routes[str(droneInConflict[count])][2][i] = round(temp1 - z_offset,1)
    routes = setZtoTarget(routes,i,solvedConflict)
routes

In [ ]:
for j in range(numDrones - 1):
    for k in range(j+1, numDrones):
        # Check next position
        if conflictXY(routes,j,k,i):
            temp = routes[str(j)][2][i]
            if conflictZ(routes,j,k,i):
                # Change z-value of the one with higher ID
                routes[str(k)][2][i] += z_offset
                routes[str(k)][2][i] = round(routes[str(k)][2][i],1)
                if (i < maxLength - 1 and checkNextXYLoc(routes,k,i+1)):
                        routes[str(k)][2][i+1] = routes[str(k)][2][i]
                temp = routes[str(k)][2][i]
            # Count how many other drones wants to occupy the same cell
            count = 1
            greater = 0
            for l in range(k+1, numDrones):
                if conflictXY(routes,j,l,i):
                    count += 1
                    # If more thant 3 drones want to access the same cell,
                    # from the 4-th and on they have to wait
                    if count > 2:
                        routes[str(l)][0].insert(i,routes[str(l)][0][i-1])
                        routes[str(l)][1].insert(i,routes[str(l)][1][i-1])
                        routes[str(l)][2].insert(i,routes[str(l)][2][i-1])
                    # Else change the altitude accordingly to the number of drones
                    # that want to access that cell
                    elif count % 2 == 0:
                        if temp > z_target:
                            # greater = 1
                            routes[str(l)][2][i] = round(temp - count*z_offset,1)
                        elif temp < z_target:
                            routes[str(l)][2][i] = round(temp + count*z_offset,1)
                            # greater = -1
                        else:
                            routes[str(l)][2][i] = round(temp - count*z_offset,1)
                        if (i < maxLength - 1 and checkNextXYLoc(routes,l,i+1)):
                            routes[str(l)][2][i+1] = round(routes[str(l)][2][i],1)
                        temp = routes[str(l)][2][i]
                    else:
                        if greater == 1:
                            routes[str(l)][2][i] = round(temp - z_offset,1)
                        elif greater == -1:
                            routes[str(l)][2][i] = round(temp + z_offset,1)
                        else:
                            routes[str(l)][2][i] = round(temp + count*z_offset,1)
                        if (i < maxLength -1 and checkNextXYLoc(routes,l,i+1)):
                            routes[str(l)][2][i+1] = round(routes[str(l)][2][i],1)
                        temp = routes[str(l)][2][i]

In [ ]:
mapBounds = [10,10]
arr = [0 if i>0 and i<mapBounds[0]-1 and j>0 and j<mapBounds[1]-1 else float('inf') for i in range(mapBounds[0]) for j in range(mapBounds[1])]
map1 = np.asarray(arr).reshape(10, 10)
map1

In [ ]:
from math import fabs
from pycrazyswarm import Crazyswarm
import numpy as np
import random
from time import sleep

swarm = Crazyswarm(map1)